## Leaky integrated firing neuron model  
## Few feautures :
### Spatial network, no memory term but exponential decay of cumulative dose

This is a minor change of the Dodds and Watts model  
Implement adaptive rewiring and generate statistics

In [20]:
#importing the required libraries

import networkx as nx
import matplotlib.pyplot as plt #for plotting
import numpy as np #for using arrays and vectorising the code wherever possible
import scipy
import random
# from numba import jit #numba precomplier to make the code faster
import pandas as pd
import copy
import warnings
warnings.filterwarnings('ignore')

# Variables used in the simulation

In [21]:
N = 100
gamma = 1 #decay of cumulative dose per unit time (second)
gamma = gamma
T = 100 #total number of seconds
dt = 0.05 #seconds
'''number of simulation steps is T/dt'''

'''Dodds watts parameters'''
p = 1 #rate of dose transfer (per unit time (second))
p = p*dt #rate of dose transfer per unit simulation step
r = 1/dt
r = r*dt
rho = 1/dt
rho = rho*dt
# r = 0.5
# r = r*dt

d_star = 1
# dose_quantity = 1
D_ini = 3

'''network parameters'''
circ_rad = 100 #for a network in which each node is connected to other nodes withing a circle of radius circ_rad
nn = 10 #the number of nearest neighbours(i.e. k) in knn
average_degree = 6
total_edges = N*2

'''rewiring parameters'''
rew_r = 18
density = 1/N
w_minus = 1
w_minus = w_minus*dt
# p_rew = 5 #rate of rewiring(per unit time(second))
# p_rew = p_rew*dt

# Creating the network. 
### The network outputs coordinates, distance matrix, edge_list and adjacency matrix

In [22]:
'''
obtains the number of nodes N
generates N points
calculates euclidean distance between each pair of points
returns the coordinates of the points and the distance matrix which is N*N
'''
def calc_dist(N) :
    coords = []
    for counter in range(N) :
        coords.append((N*np.random.random(),N*np.random.random()))

    dist_mat = scipy.spatial.distance.cdist(coords,coords)
    return dist_mat,coords

### 1. metric network

In [23]:
'''network = connect to points withing a circle of radius'''
#creates a link between points/nodes which satisfies the conditions of the network
def coupling(dist_mat,circ_rad) :
    edge_list = []
    edge_mask = (dist_mat<circ_rad) & (dist_mat>0)
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(edge_mask))))
#     print(type(np.ndarray.tolist(np.transpose(np.where(edge_mask)))))
    return edge_list

### 2. K nearest neighbour network

In [24]:
'''network = knn'''
def knn(dist_mat,nn,N) :
    near_neigh = np.argsort(dist_mat)
    selec_near_neigh = np.zeros((N,nn))
    selec_near_neigh = near_neigh[:,0:nn+1]

    edge_list = []
    for i in range(N) :
        for j in range(1,nn+1) :
            link = [i,selec_near_neigh[i,j]]
            edge_list.append(link)

    return edge_list

### 3. Random Network : GNP type

In [25]:
'''random network'''
def rand_network(average_degree,N) :
    z1 = np.random.uniform(size = (N,N))
    E,F = np.meshgrid(np.arange(0,N),np.arange(0,N))
    mask = ((average_degree/N) > z1) & (E!=F)
    adjacency_matrix = np.int64(np.zeros(shape=(N,N)))
    adjacency_matrix[mask] = np.int64(1)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list,adjacency_matrix

### 4. Random Network : GNM type

In [26]:
def rand_net_gnm(total_edges,N) :
    the_graph = nx.gnm_random_graph(N, total_edges,directed=True)
    adjacency_matrix = nx.adjacency_matrix(the_graph)
    adjacency_matrix = np.asarray(adjacency_matrix.todense())
    np.fill_diagonal(adjacency_matrix,0)
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adjacency_matrix==1))))
    return edge_list

# Function defintions required for the main part of the script

In [27]:
# # @jit(nopython=True)
# '''infecting the left part of space.'''
# def left_part_infec(N) :
#     x_coord = []
#     y_coord = []
#     for j in range(len(coords)) :
#         x_coord.append(coords[j][0])
#         y_coord.append(coords[j][1])
#     x_coord = np.asarray(x_coord)
#     y_coord = np.asarray(y_coord)
#     points = np.asarray(np.where(x_coord<(N/5))) 
#     indi_state[points.T] = 2

#     return indi_state

In [28]:
'''infecting connected nodes'''
def InfectNetworkNeighbors(net,seed_node,init_infected_nodes):
     # if in bulk find one node randomly, and infect its neighbours
    infected_nodes = set()
    candidate_nodes = set()
    explored_nodes = set()

    #pick the seed node
    infected_nodes.add(seed_node)
    explored_nodes.add(seed_node)

    curr_node=seed_node

    #add its neighbors to the list of candidates
    for n in net.neighbors(curr_node):
        candidate_nodes.add(int(n))
    #print( curr_node)
    #print( candidate_nodes)

    #while we need to select more nodes...
    while len(infected_nodes) < init_infected_nodes:

        #if there are candidate nodes, select one of them
        if(len(candidate_nodes) > 0):
            new_node = np.random.choice(list(candidate_nodes),1)[0]
            infected_nodes.add(new_node)
            candidate_nodes.remove(new_node)

        elif len(infected_nodes - explored_nodes) > 0:
            curr_node = np.random.choice(list(infected_nodes -
            explored_nodes),1)[0]
            explored_nodes.add(curr_node)
            for n in set(net.neighbors(curr_node)) - infected_nodes:
                candidate_nodes.add(n)

        else:
#             print('Initial node infection step failed')
            return None
    return infected_nodes

In [29]:
''' function to transfer doses'''
# @jit(nopython=True)
def dose(adj_mat,p,partner_state_prev,d) :

    z1 = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    dose_transfer_mask = p > z1
    mod_adj_mat = np.multiply(adj_mat,z1) #modifying adjacency matrix to include the proability of dose transfer
    dose_mask = (p>mod_adj_mat) & (adj_mat!=0) & (partner_state_prev == 2)
    d[dose_mask] = dose_quantity #whenever it is proabable, dose transfer occurs
    
    return d

In [30]:
def rew(p_rew,adjacency_matrix,indi_state_for_rew_prev,OD_mesh2,lamb_da,density,rew_r) :
    
    z1_rew_prew = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    z1_rew_lambda = np.random.uniform(size=(len(adj_mat),len(adj_mat[0])))
    
    rew_mask_plus = (p_rew > z1_rew_prew)  & (indi_state_for_rew_prev == 1) & (dist_mat <= rew_r)
    #create links. Dont break any.
    adjacency_matrix[rew_mask_plus] = 1
    rew_mask_minus1 = (w_minus > z1_rew_prew) & (indi_state_for_rew_prev == 2)
    #break links. Dont create any.
    adjacency_matrix[rew_mask_minus1] = 0
    rew_mask_minus2 = ((lamb_da*OD_mesh2) > z1_rew_lambda) & (indi_state_for_rew_prev == 1)
    adjacency_matrix[rew_mask_minus2] = 0
    
    np.fill_diagonal(adjacency_matrix,0)
    
    return adjacency_matrix

In [31]:
'''function to update cumulative doses'''
def cumu_dose(d,D_prev,gamma) :
    I = d.sum(axis=1).reshape(N,1)
#     I = 0
    D = (D_prev - (gamma*D_prev*dt)) + I
    return D

In [32]:
'''function to update the states of the nodes of the system'''
def upd_indi_state(D,d_star,indi_state_prev) :
    
    z2 = np.random.uniform(size=(N,1))
    z3 = np.random.uniform(size=(N,1))
    
    indi_state = indi_state_prev
    
    indi_state_mask1 = (D>=d_star) & (indi_state_prev==1)
    indi_state[indi_state_mask1] = 2
    
    indi_state_mask21 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho>=z3)
    indi_state[indi_state_mask21] = 1
    
    indi_state_mask22 = (D<d_star) & (indi_state_prev==2) & (r>=z2) & (rho<z3)
    indi_state[indi_state_mask22] = 3
    
    indi_state_mask23 = (D<d_star) & (indi_state_prev==2) & (r<z2)
    indi_state[indi_state_mask23] = 2
    
    return indi_state

In [33]:
'''function to get/identify the infected nodes and the suceptible nodes'''
def states(indi_state) :
    infec_indi = []
    suscep_indi = []
    infec_indi = np.transpose(np.where(indi_state==2))
    suscep_indi = np.transpose(np.where(indi_state==1))
    return infec_indi,suscep_indi

In [34]:
def networkx_graph(coords,edge_list) :
    G = nx.DiGraph()
    pos = {(i): (coords[i][0],coords[i][1]) for i in range(N)}
    G.add_nodes_from(pos.keys())
    G.add_edges_from(edge_list)
    return G

In [35]:
def edge_from_adj_mat(adj_mat_list,q):
    edge_list = []
    edge_list.extend(np.ndarray.tolist(np.transpose(np.where(adj_mat_list[q]==1))))
    return edge_list

# The main part of the script

In [36]:
#infected state time series data frame
#columns indicate the time steps
timeseries_infec_frac = pd.DataFrame()

#in degree as columns and rows as nodes
timeseries_in_degree = pd.DataFrame() 

#out degree as columns and rows as nodes
timeseries_out_degree = pd.DataFrame()

#strongly connected components as columns
timeseries_connec_comps = pd.DataFrame()

In [37]:
vals = round(1/((density)*(np.pi)*(rew_r*rew_r)),3)
p_rew_vals = [round(vals,3),round(10*vals,3),round(100*vals,3),round(1000*vals,3)]
start_vals = [1,10]
dose_quantity_vals = [0.5,2]

In [38]:
for sim in range(70,70+100) :
    '''returns coordinates, distance matrix, edge_list and the adjacency matrix'''
    dist_mat,coords = calc_dist(N) #node placement
    edge_list = knn(dist_mat,nn,N)
    org_adj_mat = np.int64(np.zeros((N,N)))
    for i in range(len(edge_list)):
        org_adj_mat[edge_list[i][0],edge_list[i][1]] = np.int64(1)
        
    df_row = 0
    for p_rew in p_rew_vals : #rate of rewiring (per unit time (second))
        lamb_da = p_rew/10
        lamb_da = lamb_da*dt
        p_rew = p_rew*dt
        for start in start_vals : #rate of dose transfer (per unit time (second))
            for dose_quantity in dose_quantity_vals : #number of nodes to infect initially
                print('(sim,lamb_da,p_rew,start,dose_quantity) = ', (sim,lamb_da/dt,p_rew/dt,start,dose_quantity))

                '''clearing exsiting data frames and creating new ones '''
                timeseries_infec_frac = pd.DataFrame()
                timeseries_in_degree = pd.DataFrame()
                timeseries_out_degree = pd.DataFrame()
                timeseries_connec_comps = pd.DataFrame()

                q = 0 #to generate new edge_list from the new adjacency matrix 
                t = np.arange(0,T,dt)

    #             numpy_array = np.zeros((N,len(t)),dtype = np.int64) #stores the states of the nodes. Rows are nodes and columns are time steps
                D_array = np.zeros((N,len(t))) #stores the cumulative doses. Rows are nodes and columns are time steps

                adj_mat_list = []
                adj_mat = copy.deepcopy(org_adj_mat)
                adj_mat_new = copy.deepcopy(adj_mat)
                adj_mat_list.append(adj_mat_new) #list of arrays which shows the time series of the adjacency matrix

                #coords remain the same. Get new edge_list from the latest adjacency matrix
                edge_list = edge_from_adj_mat(adj_mat_list,q)
                G = networkx_graph(coords,edge_list) #networkx graph

                '''from G, obtain the in degree and the out degree'''
                timeseries_in_degree.loc[:,0] = np.asarray([val for (node, val) in G.in_degree()])
                timeseries_out_degree.loc[:,0] = np.asarray([val for (node, val) in G.out_degree()])

                '''strongly and weekely connected components'''
                timeseries_connec_comps.loc[0,0] = nx.number_connected_components(G.to_undirected())

                '''choosing initially infected nodes'''
                indi_state = np.random.randint(1,2,size=(N,1))
                infected_nodes = None
                while infected_nodes is None : #infecting 'start' number of network neighbours
                    infected_nodes = InfectNetworkNeighbors(G,np.random.randint(N),start)      
                print(infected_nodes)
                infected_nodes = np.asarray(list(infected_nodes)).reshape(len(infected_nodes),1)
                indi_state[infected_nodes[:,0],0] = 2

                A,B = np.meshgrid(indi_state,indi_state) 
                partner_state = copy.deepcopy(A)
                indi_state_for_rew = copy.deepcopy(B)

    #             numpy_array [:,0] = indi_state[:,0]

                indi_state_prev = copy.deepcopy(indi_state)
                partner_state_prev = copy.deepcopy(partner_state)
                indi_state_for_rew_prev = copy.deepcopy(indi_state_for_rew)

                d = np.zeros((len(adj_mat),len(adj_mat[0])))

                D = np.zeros((N,1))
                D[np.where(indi_state==2)] = D_ini
                D_array[:,0] = D[:,0]
                D_prev = copy.deepcopy(D)

                infec_frac = np.count_nonzero(indi_state == 2)/N
                timeseries_infec_frac.loc[df_row,0] = infec_frac
                '''the main part of the simulation'''
                counter = 0
                df_col = 1
                for t in np.arange(dt,T,dt) :
                    q = q + 1 
                    counter = counter + 1
                    infec_indi = []
                    suscep_indi = []
                    d = np.zeros((len(adj_mat),len(adj_mat[0])))
                    d = dose(adj_mat,p,partner_state_prev,d)

                    OD_array = np.asarray([val for (node, val) in G.out_degree()])
                    OD_mesh1,OD_mesh2 = np.meshgrid(OD_array,OD_array)
                    adj_mat = rew(p_rew,adj_mat,indi_state_for_rew_prev,OD_mesh2,lamb_da,density,rew_r)
                    adj_mat_new = copy.deepcopy(adj_mat)
                    adj_mat_list.append(adj_mat_new)

                    #coords remain the same. Get new edge_list from the latest adjacency matrix
                    edge_list = edge_from_adj_mat(adj_mat_list,q)
                    G = networkx_graph(coords,edge_list) #networkx graph

                    '''from G, obtain the in degree and the out degree'''
                    timeseries_in_degree.loc[:,df_col] = np.asarray([val for (node, val) in G.in_degree()])
                    timeseries_out_degree.loc[:,df_col] = np.asarray([val for (node, val) in G.out_degree()])

                    '''strongly and weekely connected components'''
                    timeseries_connec_comps.loc[df_row,df_col] = nx.number_connected_components(G.to_undirected())

                    D = cumu_dose(d,D_prev,gamma)
                    D_array[:,counter] = D[:,0]

                    indi_state = upd_indi_state(D,d_star,indi_state_prev)
                    infec_frac = np.count_nonzero(indi_state == 2)/N
                    timeseries_infec_frac.loc[df_row,df_col] = infec_frac

    #                 numpy_array[:,counter] = indi_state[:,0]
                    A,B = np.meshgrid(indi_state,indi_state)

                    infec_indi, suscep_indi = states(indi_state)

                    partner_state = copy.deepcopy(A)
                    indi_state_for_rew = copy.deepcopy(B)
                    indi_state_prev = copy.deepcopy(indi_state)
                    partner_state_prev = copy.deepcopy(partner_state)
                    indi_state_for_rew_prev = copy.deepcopy(indi_state_for_rew)

                    D_prev = copy.deepcopy(D)
                    D = np.zeros((N,1))
                    df_col = df_col + 1 

    #             df_row = df_row + 1 #going to the next row of the df to store timeseries of next (p,start)

                filename_infec_frac = 'infec_frac.h5'
                filename_in_deg = 'in_deg.h5'
                filename_out_deg = 'out_deg.h5'
                filename_connec_comps = 'connec_comps.h5'
                p_rew_val = '%g'%(p_rew/dt)
                key_val = 'sim_'+str(sim)+'_p_rew_'+str(p_rew_val)+'_start_'+str(start)+'_dq_'+str(dose_quantity)
#                 print(key_val)
                timeseries_infec_frac.to_hdf(filename_infec_frac, key = key_val, mode='a')
                timeseries_in_degree.to_hdf(filename_in_deg, key = key_val, mode='a')
                timeseries_out_degree.to_hdf(filename_out_deg, key = key_val, mode='a')
                timeseries_connec_comps.to_hdf(filename_connec_comps, key = key_val, mode='a')
                df_row = 0

(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0098, 0.098, 1, 0.5)
{94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0098, 0.098, 1, 2)
{94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0098, 0.098, 10, 0.5)
{69, 8, 11, 44, 14, 48, 50, 87, 25, 27}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.0098, 0.098, 10, 2)
{43, 46, 82, 85, 86, 90, 92, 61, 30, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.098, 0.98, 1, 0.5)
{14}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.098, 0.98, 1, 2)
{38}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.098, 0.98, 10, 0.5)
{98, 7, 40, 74, 18, 53, 23, 56, 91, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.098, 0.98, 10, 2)
{65, 67, 71, 42, 78, 52, 21, 88, 89, 93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.9800000000000001, 9.8, 1, 0.5)
{11}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.9800000000000001, 9.8, 1, 2)
{57}
(sim,lamb_da,p_rew,start,dose_quantity) =  (70, 0.9800000000000001, 9.8, 10, 0.5)
{64, 97, 0

(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.9800000000000001, 9.8, 1, 0.5)
{91}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.9800000000000001, 9.8, 1, 2)
{29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.9800000000000001, 9.8, 10, 0.5)
{66, 35, 36, 5, 45, 78, 14, 48, 90, 28}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 0.9800000000000001, 9.8, 10, 2)
{32, 67, 68, 71, 44, 82, 54, 87, 61, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 9.8, 98.0, 1, 0.5)
{3}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 9.8, 98.0, 1, 2)
{65}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 9.8, 98.0, 10, 0.5)
{35, 5, 39, 14, 78, 60, 48, 55, 90, 28}
(sim,lamb_da,p_rew,start,dose_quantity) =  (75, 9.8, 98.0, 10, 2)
{0, 66, 36, 76, 45, 80, 21, 23, 58, 27}
(sim,lamb_da,p_rew,start,dose_quantity) =  (76, 0.0098, 0.098, 1, 0.5)
{52}
(sim,lamb_da,p_rew,start,dose_quantity) =  (76, 0.0098, 0.098, 1, 2)
{66}
(sim,lamb_da,p_rew,start,dose_quantity) =  (76, 0.0098, 0.098, 10, 0.5)
{64, 35, 6

(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.0098, 0.098, 1, 0.5)
{2}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.0098, 0.098, 1, 2)
{94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.0098, 0.098, 10, 0.5)
{68, 69, 40, 9, 11, 46, 47, 82, 86, 27}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.0098, 0.098, 10, 2)
{0, 66, 38, 73, 75, 76, 80, 23, 28, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.098, 0.98, 1, 0.5)
{13}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.098, 0.98, 1, 2)
{93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.098, 0.98, 10, 0.5)
{32, 97, 34, 65, 35, 5, 44, 77, 15, 81}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.098, 0.98, 10, 2)
{2, 34, 58, 35, 70, 26, 13, 87, 90, 93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.9800000000000001, 9.8, 1, 0.5)
{77}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.9800000000000001, 9.8, 1, 2)
{19}
(sim,lamb_da,p_rew,start,dose_quantity) =  (81, 0.9800000000000001, 9.8, 10, 0.5)
{34, 2, 58, 9

(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.9800000000000001, 9.8, 1, 0.5)
{0}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.9800000000000001, 9.8, 1, 2)
{97}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.9800000000000001, 9.8, 10, 0.5)
{0, 97, 7, 39, 41, 44, 77, 27, 92, 61}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 0.9800000000000001, 9.8, 10, 2)
{64, 35, 12, 79, 47, 22, 86, 57, 95, 63}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 9.8, 98.0, 1, 0.5)
{9}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 9.8, 98.0, 1, 2)
{85}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 9.8, 98.0, 10, 0.5)
{65, 66, 33, 3, 40, 75, 13, 15, 48, 26}
(sim,lamb_da,p_rew,start,dose_quantity) =  (86, 9.8, 98.0, 10, 2)
{32, 33, 65, 2, 40, 75, 13, 15, 85, 60}
(sim,lamb_da,p_rew,start,dose_quantity) =  (87, 0.0098, 0.098, 1, 0.5)
{18}
(sim,lamb_da,p_rew,start,dose_quantity) =  (87, 0.0098, 0.098, 1, 2)
{19}
(sim,lamb_da,p_rew,start,dose_quantity) =  (87, 0.0098, 0.098, 10, 0.5)
{32, 34, 67,

(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.0098, 0.098, 1, 0.5)
{66}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.0098, 0.098, 1, 2)
{64}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.0098, 0.098, 10, 0.5)
{6, 71, 40, 13, 46, 79, 47, 86, 55, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.0098, 0.098, 10, 2)
{67, 77, 14, 81, 18, 51, 19, 22, 27, 93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.098, 0.98, 1, 0.5)
{21}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.098, 0.98, 1, 2)
{96}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.098, 0.98, 10, 0.5)
{0, 7, 42, 17, 82, 49, 54, 57, 28, 30}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.098, 0.98, 10, 2)
{1, 33, 97, 68, 41, 10, 44, 84, 86, 56}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.9800000000000001, 9.8, 1, 0.5)
{81}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.9800000000000001, 9.8, 1, 2)
{9}
(sim,lamb_da,p_rew,start,dose_quantity) =  (92, 0.9800000000000001, 9.8, 10, 0.5)
{0, 38, 7, 17

...

(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.098, 0.98, 1, 2)
{83}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.098, 0.98, 10, 0.5)
{98, 66, 69, 72, 75, 78, 29, 23, 88, 93}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.098, 0.98, 10, 2)
{96, 3, 6, 71, 49, 53, 21, 24, 57, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.9800000000000001, 9.8, 1, 0.5)
{22}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.9800000000000001, 9.8, 1, 2)
{98}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.9800000000000001, 9.8, 10, 0.5)
{5, 11, 44, 45, 60, 92, 47, 51, 25, 28}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 0.9800000000000001, 9.8, 10, 2)
{1, 65, 34, 6, 71, 15, 49, 56, 27, 62}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 9.8, 98.0, 1, 0.5)
{63}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 9.8, 98.0, 1, 2)
{18}
(sim,lamb_da,p_rew,start,dose_quantity) =  (146, 9.8, 98.0, 10, 0.5)
{65, 2, 35, 34, 1, 71, 76, 14, 15, 27}
(sim,lamb_da,p_rew,start,dose_quantity) 

(sim,lamb_da,p_rew,start,dose_quantity) =  (151, 9.8, 98.0, 1, 0.5)
{78}
(sim,lamb_da,p_rew,start,dose_quantity) =  (151, 9.8, 98.0, 1, 2)
{51}
(sim,lamb_da,p_rew,start,dose_quantity) =  (151, 9.8, 98.0, 10, 0.5)
{65, 34, 4, 70, 27, 49, 52, 86, 59, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (151, 9.8, 98.0, 10, 2)
{72, 9, 74, 73, 45, 13, 14, 82, 24, 89}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.0098, 0.098, 1, 0.5)
{15}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.0098, 0.098, 1, 2)
{76}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.0098, 0.098, 10, 0.5)
{38, 41, 14, 47, 16, 79, 27, 61, 30, 95}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.0098, 0.098, 10, 2)
{78, 15, 80, 51, 84, 20, 21, 23, 88, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.098, 0.98, 1, 0.5)
{0}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.098, 0.98, 1, 2)
{64}
(sim,lamb_da,p_rew,start,dose_quantity) =  (152, 0.098, 0.98, 10, 0.5)
{10, 45, 15, 50, 83, 84, 18, 22, 23, 51}
(si

(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.0098, 0.098, 10, 2)
{64, 96, 2, 70, 41, 12, 54, 89, 58, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.098, 0.98, 1, 0.5)
{63}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.098, 0.98, 1, 2)
{83}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.098, 0.98, 10, 0.5)
{65, 98, 68, 69, 39, 73, 77, 50, 84, 57}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.098, 0.98, 10, 2)
{0, 33, 38, 11, 76, 13, 55, 56, 28, 30}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.9800000000000001, 9.8, 1, 0.5)
{0}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.9800000000000001, 9.8, 1, 2)
{32}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.9800000000000001, 9.8, 10, 0.5)
{97, 8, 40, 42, 82, 90, 59, 92, 95, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 0.9800000000000001, 9.8, 10, 2)
{1, 41, 14, 15, 16, 81, 52, 23, 27, 94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (157, 9.8, 98.0, 1, 0.5)
{2}
(sim,lamb_da,p_rew,start,dose_qua

(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 0.9800000000000001, 9.8, 10, 0.5)
{64, 34, 6, 76, 13, 85, 56, 25, 26, 92}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 0.9800000000000001, 9.8, 10, 2)
{98, 34, 13, 14, 80, 85, 88, 25, 56, 92}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 9.8, 98.0, 1, 0.5)
{11}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 9.8, 98.0, 1, 2)
{70}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 9.8, 98.0, 10, 0.5)
{64, 34, 66, 6, 8, 76, 16, 85, 57, 26}
(sim,lamb_da,p_rew,start,dose_quantity) =  (162, 9.8, 98.0, 10, 2)
{33, 97, 3, 9, 73, 15, 18, 82, 22, 61}
(sim,lamb_da,p_rew,start,dose_quantity) =  (163, 0.0098, 0.098, 1, 0.5)
{56}
(sim,lamb_da,p_rew,start,dose_quantity) =  (163, 0.0098, 0.098, 1, 2)
{13}
(sim,lamb_da,p_rew,start,dose_quantity) =  (163, 0.0098, 0.098, 10, 0.5)
{38, 6, 76, 84, 86, 56, 58, 59, 92, 94}
(sim,lamb_da,p_rew,start,dose_quantity) =  (163, 0.0098, 0.098, 10, 2)
{70, 38, 27, 6, 76, 84, 56, 59, 92, 94}
(sim,lamb_da,p_rew,st

(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.0098, 0.098, 1, 2)
{48}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.0098, 0.098, 10, 0.5)
{97, 35, 5, 10, 83, 51, 25, 58, 94, 31}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.0098, 0.098, 10, 2)
{34, 68, 8, 45, 83, 51, 53, 20, 23, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.098, 0.98, 1, 0.5)
{96}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.098, 0.98, 1, 2)
{6}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.098, 0.98, 10, 0.5)
{96, 64, 4, 75, 14, 78, 50, 86, 57, 61}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.098, 0.98, 10, 2)
{68, 9, 10, 44, 45, 15, 82, 51, 85, 29}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.9800000000000001, 9.8, 1, 0.5)
{56}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.9800000000000001, 9.8, 1, 2)
{42}
(sim,lamb_da,p_rew,start,dose_quantity) =  (168, 0.9800000000000001, 9.8, 10, 0.5)
{97, 67, 35, 42, 10, 43, 26, 92, 94, 31}
(sim,lamb_da,p_rew,start,dose_quantity